<h1>CNNs for EPRV</h1>
The goal here is to training a CNN using HARPS images to the outputs of the HARPS EPRV extraction pipeline to see it a large of NN can replicated more explicit modeling.

In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.transforms import transforms
from torch.utils.data import DataLoader

<h2>Model definition</h2>

In [165]:
class DownSizeNet(nn.Module):
    def __init__(self, in_size, out_size, normalize=True, leaky_slope=0.2):
        super(DownSizeNet, self).__init__()
        layers = [nn.Conv2d(in_size, out_size, 3, stride=2, padding=1, bias=False).double()]
        if normalize:
            layers.append(nn.BatchNorm2d(out_size, 0.8).double())
        layers.append(nn.LeakyReLU(leaky_slope).double())
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

class RV_Model(nn.Module):
    def __init__(self):
        super(RV_Model, self).__init__()
#         channels_in, self.h, self.w = in_shape
#         channels_out, _, _ = out_shape

#         self.fc    = nn.Linear(latent_dim, self.h * self.w)

        self.down1 = DownSizeNet(3, 64, normalize=False)
        self.down2 = DownSizeNet(64, 128)
        self.down3 = DownSizeNet(128, 256)
        self.down4 = DownSizeNet(256, 512)
        self.down5 = DownSizeNet(512, 512)
        self.down6 = DownSizeNet(512, 512)
#         self.down7 = DownSizeNet(512, 1, normalize=False)
        
        self.final = nn.Sequential(
            nn.Conv2d(512, 1, 3, stride=1, padding=1).double(), nn.Tanh().double()
        )
        
        
    def forward(self, x):
        # Propogate noise through fc layer and reshape to img shape
#         z = self.fc(z).view(z.size(0), 1, self.h, self.w)
        d1 = self.down1(x)
#         print('d1: {}'.format(d1.shape))
        d2 = self.down2(d1)
#         print('d2: {}'.format(d2.shape))
        d3 = self.down3(d2)
#         print('d3: {}'.format(d3.shape))
        d4 = self.down4(d3)
#         print('d4: {}'.format(d4.shape))
        d5 = self.down5(d4)
#         print('d5: {}'.format(d5.shape))
        d6 = self.down6(d5)
#         print('d6: {}'.format(d6.shape))
#         d7 = self.down7(d6)
#         print('d7: {}'.format(d7.shape))
        
        
        return self.final(d6)

In [128]:
from torch.utils.data import Dataset
import glob

In [129]:
from astropy.io import fits

<h2>Dataset importing</h2>
The question here is how is the data organized in the directory and how can it be imported with the target RV. 

Not the OG data but after the data is saved from the pre processing step.

In [8]:
import pickle

In [9]:
def load_data(filename):
      with open(filename, 'rb') as handle:
            obj = pickle.load(handle)
            return obj

In [10]:
xs_train = load_data('../data/xs_train.pickle')
ys_train = load_data('../data/ys_train.pickle')
xs_valid = load_data('../data/xs_valid.pickle')
ys_valid = load_data('../data/ys_valid.pickle')

In [116]:
class RV_Dataset(Dataset):
    def __init__(self, xs_data, ys_data):
       
        # NO RESIZE
        self.transform = transforms.ToTensor()#transforms.Compose(

        print(self.transform,type(self.transform))
        self.data = {'x':xs_data,'y':ys_data}

    def __getitem__(self, index):
        
        return self.transform(self.data['x'][index]).to(torch.double), self.data['y'][index]
    
    
    def __len__(self):
        return len(self.data['y'])


In [112]:
from astropy.nddata import Cutout2D
from astropy import units as u

In [105]:
import os.path

<h2>Preprocess</h2>
take in the FITS images and chop them up to sizes small enough to fit on a GPU with the model. Then these files are fed into the Dataset above.

In [92]:
def preprocess(file_names,outdir,size):
    files = sorted(glob.glob(file_names))
    for file_name in files:
        tbl = fits.open(file_name)
        img_shape = tbl[0].data.shape
        integer = (img_shape[1]//size) + 1
        delta_x = (img_shape[1] - size)/integer
        for i in range(integer):
            position = (img_shape[0]/2, size/2 + i * delta_x)
            size     = (shape[0], size)
            sci   = Cutout2D(tbl['sci'], position, size)
            ref   = Cutout2D(tbl['ref'], position, size)
            
            sci_hdu = fits.ImageHDU(data=sci)
            ref_hdu = fits.ImageHDU(data=ref)
            
            hdu = fits.HDUList([sci_hdu,ref_hdu])
            name = path.join(outdir,path.split(file_name)[1] + '_c{}.fits'.format(i))
            hdu.header['RV'] = tbl['RV']
            hdu.writeto(name)

<h2>Defining Fitting Process</h2>
including hyperparameters, the loss function, and the optimization algo

In [93]:
lr = 0.001#, betas=(, ),
b1 = 0.9
b2 = 0.999

In [166]:
model = RV_Model()
mse_loss = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(b1, b2))

In [167]:
xs_train.shape, ys_train.shape

((100, 128, 512, 3), (100,))

In [168]:
# directory = something
batch_size = 2
n_cpu = 1
dataloader = DataLoader(
    RV_Dataset(xs_train,ys_train),
    batch_size=batch_size,
    shuffle=True,
    num_workers=n_cpu,
)

validloader = DataLoader(
    RV_Dataset(xs_valid,ys_valid),
    batch_size=batch_size,
    shuffle=True,
    num_workers=n_cpu,
)

ToTensor() <class 'torchvision.transforms.transforms.ToTensor'>
ToTensor() <class 'torchvision.transforms.transforms.ToTensor'>


In [169]:
import time
import datetime

<h2>Training Step</h2>
the working step!

In [170]:
import sys

In [171]:
prev_time = time.time()
n_epochs = 1
validiter = iter(validloader)
for epoch in range(n_epochs):
    for i, batch in enumerate(dataloader):
        def into_func():
            imgs, target_rv = batch

            optimizer.zero_grad()
#             print('imgs: {}'.format(imgs.shape))
            something = model(imgs)
#             print('something: {}'.format(something.shape))
            y = torch.mean(something)
#             print('y: {} {}'.format(y.shape,y.dtype))
#             print('target_rv: {} {}'.format(target_rv.shape,target_rv.dtype))
            loss = mse_loss(y,target_rv.double())
            loss.backward()
            optimizer.step()

#             print('step {}'.format(i))

#             batches_done = epoch * len(dataloader) + i
#             batches_left = n_epochs * len(dataloader) - batches_done
#             time_left = datetime.timedelta(seconds=batches_left * (time.time() - prev_time))
#             prev_time = time.time()

            # Print log
            
        into_func()
        if i % 5 == 0:
            imgs, target_rv = validiter.next()
            optimizer.zero_grad()
            something = model(imgs)
            y = torch.mean(something)
            loss = mse_loss(y,target_rv.double())
            
            sys.stdout.write(
                "\r[Epoch %d/%d] [Batch %d/%d] [Loss: %f]"
                % (
                    epoch,
                    n_epochs,
                    i,
                    len(dataloader),
                    loss.item()
                )  
        )

[Epoch 0/1] [Batch 45/50] [Loss: 381.597336]

<h2>Slicing whole image problem</h2>
Since a whole HARPS image is 4096 pixels, 8 512x512 images could be used to exactly divide the whole. But since there is information in the over lap of any two cross sections within pixel distance less than the assiocated max velocity. Or just more than the previous known velocity precision.

c-m-c-m-c-m-c there are m divisions of image of whole size (c+m+c+m+c+m+c=3m+4c)^2 into boxes of size (c+m+c=2c+m)^2

so if m = 512, and c = 6, then 8 divisions of size 524x524, which is a total image size of 8(512-72x/8)+9(0+72x/9)=4096 = > 
8(512-9)+9(8) = 8(503) + 9(8)=4096

This first integer value is good because we can avoid interpolation, and we can rest assured that no predicted measurements exceed 8 pixels to log-wavelength

it = np.zeros(8,2)
it[0,0] = 0
it[0,1] = 2c+m = 2(8) + 503 = 16 + 503 = 519

it[i,:]=it[i-1,:]+c+m=it[i-1,:]+8+503=it[i-1,:]+511
so to iterate through this list we select, it_{i+1}= it_{i} + 2c+m

In [174]:
import numpy as np

In [177]:
m=503
c=8
itlist =  np.zeros((c,2)) 
itlist[0,0] = 0 
itlist[0,1] = 519
for i in range(1,c):
    itlist[i,:]=itlist[i-1,:]+511

In [178]:
print(itlist)

[[   0.  519.]
 [ 511. 1030.]
 [1022. 1541.]
 [1533. 2052.]
 [2044. 2563.]
 [2555. 3074.]
 [3066. 3585.]
 [3577. 4096.]]


In [184]:
m=494
c=16
n=8
itlist =  np.zeros((n,2)) 
itlist[0,0] = 0 
itlist[0,1] = m+2*c
for i in range(1,n):
    itlist[i,:]=itlist[i-1,:]+m+c

In [185]:
print(itlist)

[[   0.  526.]
 [ 510. 1036.]
 [1020. 1546.]
 [1530. 2056.]
 [2040. 2566.]
 [2550. 3076.]
 [3060. 3586.]
 [3570. 4096.]]
